In [ ]:
#================================================
# EXPLORAÇÃO INICIAL DA TABELA "ratings"
#================================================

import duckdb, pandas as pd
from pathlib import Path

# cria ou liga se já existir:
con = duckdb.connect("movielens100K.duckdb")

In [ ]:
con.sql("DESCRIBE ratings").df()


#### Comentário

 - "userID": INTEGER;
 - "movieID":  INTEGER;
 - "rating": DOUBLE
 - "timestamp":  Timestamp with time zone;

 - Coluna "null": indica se a coluna pode conter valores nulos (NULL).
 - Neste caso pode.

 - Coluna "key": indica se coluna é chave primária (PRIMARY KEY).
 - Não faz.

 - Coluna "default": indica se mostra o valor por defeito (DEFAULT).
 - Não tem

 - Coluna "extra": mostra informações adicionais sobre a coluna, como: auto_increment ou generated.
 - Neste caso não tem

In [ ]:
#Ver o tipo de dados de cada coluna
con.sql("PRAGMA table_info('ratings')").df()

In [ ]:
#Ver primeiras 10 linhas
con.sql("SELECT * FROM ratings LIMIT 10;").df()

In [ ]:
#Contar o número de missing values
con.sql("""
SELECT
    COUNT(*) - COUNT(userId)   AS missing_userId,
    COUNT(*) - COUNT(movieId)  AS missing_movieId,
    COUNT(*) - COUNT(rating)   AS missing_rating,
    COUNT(*) - COUNT(timestamp) AS missing_timestamp
FROM ratings
""").df()


In [ ]:
#Identificação dos valores máximos, mínimos e contagens das avaliações 
con.sql("""
SELECT
    MIN(userId)                  AS min_userId,
    MAX(userId)                  AS max_userId,
    COUNT(DISTINCT userId)       AS total_users,
    MIN(movieId)                 AS min_movieId,
    MAX(movieId)                 AS max_movieId,
    COUNT(DISTINCT movieId)      AS total_movies,
    MIN(rating)                  AS min_rating,
    MAX(rating)                  AS max_rating,
    AVG(rating)                  AS med_rating,
    MIN(timestamp)               AS min_timestamp,
    MAX(timestamp)               AS max_timestamp,
    COUNT(*)                     AS total_ratings
        
FROM ratings
""").df()

#### Comentários
 - Temos 610 userID a comentar.
 - 9724 movies evaluated
 - Rating between 0,5 and 5,0 and mean value of 3,50155
 - O time Stamp mais antigo de uma classificação foi em 29 Marços de 1996 às 19.36.55s
 - Timestamp mais recente em 2018 24 de Setembro às 15:27 e 30 minutos.
 - Temos um total de 100836 ratings (linhas).

In [ ]:
#Número de ratings e nota média por utilizador
con.sql("""
SELECT
    userId,
    COUNT(*)              AS total_ratings,
    ROUND(AVG(rating), 2) AS media_rating
FROM ratings
GROUP BY userId
ORDER BY total_ratings DESC, media_rating DESC
""").df()



#### Comentários
 - User 414 foi o que mais classificou com 2698 ratings e um valor médio de 3.39 por rating.
 - Depois temos 6 User com 20 classificações cada um e com uma média entre 1.725 e 5.0

In [ ]:
#Classificações médias por filme ordenados da melhor para a pior nota 
con.sql("""
SELECT
    m.title,
    ROUND(AVG(r.rating), 2) AS media_rating,
    COUNT(*)                AS total_ratings
FROM ratings r
JOIN movies m USING (movieId)
GROUP BY m.title
ORDER BY media_rating DESC, total_ratings DESC
""").df()


In [ ]:
#Classificações médias por filme ordenados do mais classificado para o menor
con.sql("""
SELECT
    m.title,
    ROUND(AVG(r.rating), 2) AS media_rating,
    COUNT(*)                AS total_ratings
FROM ratings r
JOIN movies m USING (movieId)
GROUP BY m.title
ORDER BY total_ratings DESC, media_rating DESC
""").df()


#### Fechar a ligação

In [ ]:
con.close()
print("Ligação fechada.")